## 1) ML lec 5 -1, 2

### Regression 단계
   - Hypothesis: linear 관계를 가정
   - Cost: 가정에 의한 예측과 실제값의 차이를 측정
   - Gradient decent: 학습을 통해 Cost를 최소화 하는 weight 찾기

### Binary classification 은?
- Linear regression의 hypothesis인 $y = Wx + b$를 적용했을 때
    - 데이터 분포에 따라 분류가 제대로 되지 않을 수 있음. 특히 outlier에 따라 weigth가 심하게 변동하게 되어 예측 결과가 현실과 맞지 않게 된다.


- 분류 문제를 풀기 위한 첫번째 방법론으로 logistic regression!
    - hypothesis에 따른 결과값이 0과 1사이의 값을 갖기 때문에 binary classification에 유리한 형태
    
    - Hypothesis,  Cost, Gradient decent에서 Hypothesis를 로지스틱 함수로 변경하고 이에 따라 cost function의 모양이 달라지므로 cost function에 로그를 취한 후 gradient decent 방법으로 학습



In [ ]:
from IPython.display import Image

In [ ]:
# Logistic Regression의 cost function에 로그를 취하는 이유
Image('./img/cost_logistic.png')

###  Cost funtion idea의 흐름

(1)

$Cost(W) = \frac{1}{m}\sum C\;(H(x), y)$ 

Hypothesis에 로지스틱 함수 적용하고 로그를 취하면 (2)로 정리됨


(2)

$C\;(H(x), y) = -log(H(x))$ if y = 1

$C\;(H(x), y) = -log(1 - H(x))$ if y = 0

이 형태는 if문이 들어가야 하므로 python에서 한줄로 보다 쉽게 적용하기 위해 (3)의 식으로 변형

(3)

$C\;(H(x), y) = -ylog(H(x))-(1-y)log(1-H(x))$

In [ ]:
import tensorflow as tf

## 2) ML lab 5

In [ ]:
import tensorflow as tf
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

# 앞으로는 거의 항상 다변수 데이터를 다룰 것이기 때문에 shape정의에 주의!
X = tf.placeholder(tf.float32, shape=[None, 2])
y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([2, 1]), name='weight') #여기서 shape은 [변수의 갯수, label shape]
b = tf.Variable(tf.random_normal([1]), name='bias')
yhat = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(y * tf.log(yhat) + (1 - y) * tf.log(1 - yhat))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-02)
train_op = optimizer.minimize(cost)

# classification 문제이므로 예측 결과값 yhat을 그대로 사용할 수 없음
# yhat < 0.5이면 0, yhat >= 0.5 이면 1로 분류해야 함
# 이것을 구현하기 위해 tf.cast를 사용한다

pred = tf.cast(yhat > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, y), dtype=tf.float32))


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        _, _cost = sess.run([train_op, cost], feed_dict={X: x_data, y: y_data})
        if step % 1000 == 0:
            print(step, _cost)
    _yhat, _pred, _accracy = sess.run([yhat, pred, accuracy], 
                                     feed_dict={X: x_data, y: y_data})
    print("\nyhat: ", _yhat, 
          "\npredicted: ", _pred, 
          "\nAccuracy: ", _accuracy) 
        
        

### Classifying diabetes

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('./data/data-03-diabetes.csv', header=None)
df

,0,1,2,3,4,5,6,7,8
0,-0.294118,0.487437,0.180328,-0.292929,0.000000,0.001490,-0.531170,-0.033333,0
1,-0.882353,-0.145729,0.081967,-0.414141,0.000000,-0.207153,-0.766866,-0.666667,1
2,-0.058824,0.839196,0.049180,0.000000,0.000000,-0.305514,-0.492741,-0.633333,0
3,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,0.000000,1
4,0.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000,0
5,-0.411765,0.165829,0.213115,0.000000,0.000000,-0.236960,-0.894962,-0.700000,1
6,-0.647059,-0.216080,-0.180328,-0.353535,-0.791962,-0.076006,-0.854825,-0.833333,0
7,0.176471,0.155779,0.000000,0.000000,0.000000,0.052161,-0.952178,-0.733333,1
8,-0.764706,0.979899,0.147541,-0.090909,0.283688,-0.090909,-0.931682,0.066667,0
9,-0.058824,0.256281,0.573770,0.000000,0.000000,0.000000,-0.868488,0.100000,0


In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
# sample data의 숫자가 그리 많진 않으므로 queue runner를 사용하진 않음

dataset = np.loadtxt('./data/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = dataset[:, 0:-1]
y_data = dataset[:, [-1]]
print(x_data.shape, y_data.shape)

In [ ]:
# define
X = tf.placeholder(tf.float32, shape=[None, 8])
y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')
yhat = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(y * tf.log(yhat) + (1 - y) * tf.log(1-yhat))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-02)
train_op = optimizer.minimize(cost)
pred = tf.cast(yhat > 0.5, dtype=tf.float32)
acc = tf.reduce_mean(tf.cast(tf.equal(pred, y), dtype=tf.float32))

In [ ]:
# run
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20001):
        _, _cost, _acc = sess.run([train_op, cost, acc], 
                                              feed_dict={X: x_data, y: y_data})
        if i%1000==0:
            print('step: {}, cost: {}, acc: {}'
                 .format(i, _cost, _acc))
    
                    
                                          

In [20]:
import tensorflow as tf
# filename_queue를 list 형태로 만들기
filename_queue = tf.train.string_input_producer(
    ['./data/data-03-diabetes.csv'], shuffle=False, name='filename_queue')

# reader 정의 
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [[0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=759)

X = tf.placeholder(tf.float32, shape=[None, 8])
y = tf.placeholder(tf.float32, shape=[None, 1])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    X_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    print(X_batch.shape, y_batch.shape)
    print(X_batch[0])

(759, 8) (759, 1)
[-0.29411799  0.48743701  0.180328   -0.29292899  0.          0.00149028
 -0.53117001 -0.0333333 ]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled
